# Init library

In [1]:
# get parent directory for easy calling files outside of folder conatining the code file

import os
import sys
from pathlib import Path

parent_dir = Path(os.getcwd()).resolve().parent
sys.path.append(str(parent_dir))

print(parent_dir)



O:\Projects\time_series_feature_engineering


In [45]:
#!/usr/bin/env python
# coding: utf-8

import polars as pl
import polars.selectors as cs

# Read data

In [3]:
df = pl.read_excel(parent_dir / 'data/test_data_2020_202208.xlsx')

print(df.shape)

df.head(5)

(988, 4)


date,x_1,x_2,label
date,i64,i64,i64
2022-09-14,125490,169835,108957173922
2022-09-13,107326,146494,97902777699
2022-09-12,93820,130855,95690875998
2022-09-11,64274,87798,58820280634
2022-09-10,66959,90096,69852043855


# Lagged Values Features
- Values of last 2, 7, 14, 28, 30... days.
- Values of day t-1, t-4, t-7, t-10…

In [56]:
df_prep = df.sort(by='date', descending=False)

# ====================================
# Values of last 2, 7, 14, 28, 30... days.
lags = 30 # do not use data from the first 30 days for training. There will be a bunch of null values in lag_columns.
lag_step = 3 # Values of day t-1, t-4, t-7, t-10…

lag_columns = []
value_columns = ['x_1', 'x_2', 'label']

for c in value_columns:
    for i in range(1, lags+1, lag_step):
        lag_columns.append(pl.col(c).shift(i).alias(f"{c}_lag_{i}"))

df_prep = df_prep.with_columns(lag_columns)

df_prep.tail(5)


date,x_1,x_2,label,x_1_lag_1,x_1_lag_4,x_1_lag_7,x_1_lag_10,x_1_lag_13,x_1_lag_16,x_1_lag_19,x_1_lag_22,x_1_lag_25,x_1_lag_28,x_2_lag_1,x_2_lag_4,x_2_lag_7,x_2_lag_10,x_2_lag_13,x_2_lag_16,x_2_lag_19,x_2_lag_22,x_2_lag_25,x_2_lag_28,label_lag_1,label_lag_4,label_lag_7,label_lag_10,label_lag_13,label_lag_16,label_lag_19,label_lag_22,label_lag_25,label_lag_28
date,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
2022-09-10,66959,90096,69852043855,72821,54644,20778,27443,23327,53459,74548,96324,104836,82044,97114,69028,26028,36098,30643,71014,101239,127615,144794,113005,79781916805,53346288660,22707880783,42072545272,24514786645,55953437762,72984902538,82549875296,98694718554,70168022431
2022-09-11,64274,87798,58820280634,66959,59809,21103,22713,30909,45255,73745,72303,91497,77888,90096,75899,26208,28045,41434,59788,99688,97808,127341,109136,69852043855,56484560972,22803785170,31608675158,39179147425,50471183678,68946068412,63271643103,82014715552,67757852224
2022-09-12,93820,130855,95690875998,64274,54406,45267,18867,28685,31378,56506,71076,89722,108490,87798,71847,56439,23122,38464,41374,75677,94779,126754,151076,58820280634,55782383425,52145550109,19098035913,40161119780,33137300553,55921020032,57347687103,81923594072,107653746487
2022-09-13,107326,146494,97902777699,93820,72821,54644,20778,27443,23327,53459,74548,96324,104836,130855,97114,69028,26028,36098,30643,71014,101239,127615,144794,95690875998,79781916805,53346288660,22707880783,42072545272,24514786645,55953437762,72984902538,82549875296,98694718554
2022-09-14,125490,169835,108957173922,107326,66959,59809,21103,22713,30909,45255,73745,72303,91497,146494,90096,75899,26208,28045,41434,59788,99688,97808,127341,97902777699,69852043855,56484560972,22803785170,31608675158,39179147425,50471183678,68946068412,63271643103,82014715552


In [16]:
df_prep.head(7)


date,x_1,x_2,label,day,month,quarter,year,x_1_lag_1,x_1_lag_4,x_1_lag_7,x_1_lag_10,x_1_lag_13,x_1_lag_16,x_1_lag_19,x_1_lag_22,x_1_lag_25,x_1_lag_28,x_2_lag_1,x_2_lag_4,x_2_lag_7,x_2_lag_10,x_2_lag_13,x_2_lag_16,x_2_lag_19,x_2_lag_22,x_2_lag_25,x_2_lag_28,label_lag_1,label_lag_4,label_lag_7,label_lag_10,label_lag_13,label_lag_16,label_lag_19,label_lag_22,label_lag_25,label_lag_28
date,i64,i64,i64,i8,i8,i8,i16,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
2020-01-01,1204,1467,927380311,1,1,1,2020,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2020-01-02,3179,3965,4738609473,2,1,1,2020,1204,null,null,null,null,null,null,null,null,null,1467,null,null,null,null,null,null,null,null,null,927380311,null,null,null,null,null,null,null,null,null
2020-01-03,2732,3454,4065781429,3,1,1,2020,3179,null,null,null,null,null,null,null,null,null,3965,null,null,null,null,null,null,null,null,null,4738609473,null,null,null,null,null,null,null,null,null
2020-01-04,2225,2678,2276944452,4,1,1,2020,2732,null,null,null,null,null,null,null,null,null,3454,null,null,null,null,null,null,null,null,null,4065781429,null,null,null,null,null,null,null,null,null
2020-01-05,2652,3183,2497004303,5,1,1,2020,2225,1204,null,null,null,null,null,null,null,null,2678,1467,null,null,null,null,null,null,null,null,2276944452,927380311,null,null,null,null,null,null,null,null
2020-01-06,5209,6680,6364698773,6,1,1,2020,2652,3179,null,null,null,null,null,null,null,null,3183,3965,null,null,null,null,null,null,null,null,2497004303,4738609473,null,null,null,null,null,null,null,null
2020-01-07,6276,7965,6493319066,7,1,1,2020,5209,2732,null,null,null,null,null,null,null,null,6680,3454,null,null,null,null,null,null,null,null,6364698773,4065781429,null,null,null,null,null,null,null,null


# Aggregated Lagged Values Features
- Average growth of the last 2, 7, 14, 28, 30... days.
- Rolling Mean, max, min, median, standard deviation… of days t-1, t-4, t-7, t-10…

In [61]:
# ====================================
# Average growth of the last 2, 7, 14, 28, 30... days.
growth_period = 28 # growth of the last 28 days. Must match the number of days in lagged values ablove.

value_columns = ['x_1', 'x_2', 'label']

for c in value_columns:
    df_prep = df_prep.with_columns(((pl.col(f"{c}_lag_1") - pl.col(f"{c}_lag_{growth_period}")) / pl.col(f"{c}_lag_{growth_period}")).alias(f"{c}_growth_rate_{growth_period}_days"))


# ====================================
# Rolling Mean, max, min, median, standard deviation… of days t-1, t-4, t-7, t-10…, t-25, t-28
for c in value_columns:
    df_prep = df_prep.with_columns(pl.concat_list(cs.starts_with(f"{c}_lag")).list.mean().alias(f"{c}_rolling_mean"))
    df_prep = df_prep.with_columns(pl.concat_list(cs.starts_with(f"{c}_lag")).list.min().alias(f"{c}_rolling_min"))
    df_prep = df_prep.with_columns(pl.concat_list(cs.starts_with(f"{c}_lag")).list.max().alias(f"{c}_rolling_max"))
    df_prep = df_prep.with_columns(pl.concat_list(cs.starts_with(f"{c}_lag")).list.std().alias(f"{c}_rolling_std"))

    # for easy showing result
    df_prep = df_prep.drop(cs.starts_with(f"{c}_lag"))

df_prep.tail(10)

date,x_1,x_2,label,x_1_growth_rate_28_days,x_2_growth_rate_28_days,label_growth_rate_28_days,x_1_rolling_mean,x_2_rolling_mean,label_rolling_mean,x_1_rolling_min,x_1_rolling_max,x_1_rolling_std,x_2_rolling_min,x_2_rolling_max,x_2_rolling_std,label_rolling_min,label_rolling_max,label_rolling_std
date,i64,i64,i64,f64,f64,f64,f64,f64,f64,i64,i64,f64,i64,i64,f64,i64,i64,f64
2022-09-05,45267,56439,52145550109,-0.625102,-0.660514,-0.625582,57318.8,78076.2,5.6551e10,21103,91497,25868.920996,26208,127341,37145.734517,22803785170,82014715552,1.9932e10
2022-09-06,54644,69028,53346288660,-0.295762,-0.369235,-0.198628,60318.6,82043.5,5.9500e10,18867,108490,29687.025592,23122,151076,42691.247313,19098035913,107653746487,2.6154e10
2022-09-07,59809,75899,56484560972,-0.443277,-0.488325,-0.494776,63555.6,85437.0,6.2858e10,20778,104836,32268.344458,26028,144794,44972.267714,22707880783,105589463477,2.8493e10
2022-09-08,54406,71847,55782383425,-0.266012,-0.33489,-0.280955,57670.7,77946.2,5.6109e10,21103,91497,25877.304497,26208,127341,37151.419975,22803785170,82014715552,1.9874e10
2022-09-09,72821,97114,79781916805,-0.388126,-0.417172,-0.324215,59331.4,80280.5,5.8572e10,18867,108490,29704.866321,23122,151076,42714.183664,19098035913,107653746487,2.6100e10
2022-09-10,66959,90096,69852043855,-0.112415,-0.140622,0.137012,61022.4,81657.8,6.0277e10,20778,104836,30177.090307,26028,144794,41831.527998,22707880783,98694718554,2.5167e10
2022-09-11,64274,87798,58820280634,-0.140317,-0.174461,0.030907,56218.1,75544.3,5.5239e10,21103,91497,24776.254665,26208,127341,35282.662356,22803785170,82014715552,1.8951e10
2022-09-12,93820,130855,95690875998,-0.407558,-0.418849,-0.453616,56867.1,76733.0,5.6199e10,18867,108490,27947.978601,23122,151076,40142.556473,19098035913,107653746487,2.4720e10
2022-09-13,107326,146494,97902777699,-0.105078,-0.096268,-0.030436,62200.0,83442.8,6.2830e10,20778,104836,31297.530069,26028,144794,43658.709073,22707880783,98694718554,2.7470e10


# Lagged Values Features
- Values of the same date in the previous 1, 2, 3… months or quarters or years.

In [62]:
# ====================================
# Values of the same date in the previous 1, 2, 3… months or quarters or years.
# Example: date = 2022-09-01 -> same date in the previous 1 month = 2022-08-01
# The end date of a month is usually different from eachother. Can use the next or previous date to fill in: date = 2022-07-31 -> same date in the previous 1 month = 2022-07-01 or 2022-06-30.
# The choice depends on the characteristics of the data and the purpose of forecasting.
prev_month = 3 # do not use data from the first 3 months for training. There will be a bunch of null values in prev_3_month_value.
prev_step = 1

###############
# Version: Using previous date to fill in

# get previous month date for joining
prev_columns = []

for i in range(1, prev_month+1, prev_step):
    # by default: get previous date to fill in: 2022-06-30 for 2022-07-31
    prev_columns.append(pl.col("date").dt.offset_by(f"-{i}mo").alias(f"prev_{i}_month"))
    
    # https://docs.pola.rs/api/python/stable/reference/expressions/api/polars.Expr.dt.offset_by.html
    # # same date last week
    # prev_columns.append(pl.col("date").dt.offset_by(f"-{i}w").alias(f"prev_{i}_week"))
    
    # # same date last quarter
    # prev_columns.append(pl.col("date").dt.offset_by(f"-{i}q").alias(f"prev_{i}_quarter"))
    
    # # same date last year
    # prev_columns.append(pl.col("date").dt.offset_by(f"-{i}y").alias(f"prev_{i}_year"))
    
df_prep = df.with_columns(prev_columns)

# get previous month data by self-joining with date = prev_{i}_month
value_columns = ['x_1', 'x_2', 'label']

for i in range(1, prev_month+1, prev_step):
    
    value_columns_rename = {'date': 'date_join'}
    
    for c in value_columns:
        value_columns_rename[c] = f"{c}_prev_{i}_month"
    
    df_prep = df_prep.join(df_prep.select(['date'] + value_columns).rename(value_columns_rename),
                           left_on=f"prev_{i}_month",
                           right_on='date_join', # by default polars not adding right_on column to result
                           how="left")

# # delete columns with name starts with "prev_"
# df_prep = df_prep.drop(cs.starts_with("prev_"))

df_prep.slice(45, 10)

date,x_1,x_2,label,prev_1_month,prev_2_month,prev_3_month,x_1_prev_1_month,x_2_prev_1_month,label_prev_1_month,x_1_prev_2_month,x_2_prev_2_month,label_prev_2_month,x_1_prev_3_month,x_2_prev_3_month,label_prev_3_month
date,i64,i64,i64,date,date,date,i64,i64,i64,i64,i64,i64,i64,i64,i64
2022-07-31,17420,24136,20593939456,2022-06-30,2022-05-31,2022-04-30,27916,36342,39842656892,27173,39031,37684013508,16293,21353,20212177918
2022-07-30,19884,28171,26267033664,2022-06-30,2022-05-30,2022-04-30,27916,36342,39842656892,26491,36741,32970867146,16293,21353,20212177918
2022-07-29,25323,36250,34789820354,2022-06-29,2022-05-29,2022-04-29,26432,34479,33639337985,19190,27018,20033812465,27990,38629,38403618787
2022-07-28,31960,46610,39199979007,2022-06-28,2022-05-28,2022-04-28,30765,40291,38757831407,24925,35433,26607409196,27805,36605,34419342538
2022-07-27,39491,56775,44855670306,2022-06-27,2022-05-27,2022-04-27,38951,52188,46459958187,35688,50253,39256733176,35363,47754,39465848085
2022-07-26,44962,64517,50619352887,2022-06-26,2022-05-26,2022-04-26,32047,42082,32913999800,39508,56735,40585728121,39089,54358,41236016067
2022-07-25,53791,77065,60007324017,2022-06-25,2022-05-25,2022-04-25,40549,53435,41162730219,48996,68597,47676460870,47159,65156,48224694107
2022-07-24,41984,61492,38920039372,2022-06-24,2022-05-24,2022-04-24,49293,65831,49077430069,52170,73588,47992337287,35880,50460,29286705126
2022-07-23,50741,71571,47471399778,2022-06-23,2022-05-23,2022-04-23,56044,74285,51673509983,61829,87711,53451912203,46104,63838,38092320469


In [55]:
# ====================================
# Values of the same date in the previous 1, 2, 3… months or quarters or years.
# Example: date = 2022-09-01 -> same date in the previous 1 month = 2022-08-01
# The end date of a month is usually different from eachother. Can use the next or previous date to fill in: date = 2022-07-31 -> same date in the previous 1 month = 2022-07-01 or 2022-06-30.
# The choice depends on the characteristics of the data and the purpose of forecasting.
prev_month = 3 # do not use data from the first 3 months for training. There will be a bunch of null values in prev_3_month_value.
prev_step = 1

# get previous month date for joining
prev_columns = []

###############
# Version: Using next date to fill in

# ====================================
# Day of date
# For filtering the time more simple
# cast to Int8 or Int16 for smaller data size
df_prep = df.with_columns(day=pl.col('date').dt.day().cast(pl.Int8))

for i in range(1, prev_month+1, prev_step):
    # by default: get previous date to fill in: 2022-06-30 for 2022-07-31
    df_prep = df_prep.with_columns(pl.col("date").dt.offset_by(f"-{i}mo").alias(f"prev_{i}_month"))
    
    # day of prev_month
    # if day of prev_month < day of date -> use the next date of prev_month
    df_prep = df_prep.with_columns(pl.col(f"prev_{i}_month").dt.day().cast(pl.Int8).alias(f"prev_{i}_month_day"))
    
    df_prep = df_prep.with_columns((pl.col(f"day") - pl.col(f"prev_{i}_month_day")).alias(f"prev_{i}_month_day_diff"))
    
    df_prep = df_prep.with_columns(pl.when(pl.col(f"prev_{i}_month_day_diff") > 0).then(pl.col(f"prev_{i}_month") + pl.duration(days=pl.col(f"prev_{i}_month_day_diff")))
                                   .otherwise(pl.col(f"prev_{i}_month")).alias(f"prev_{i}_month"))


df_prep.slice(167, 10)

date,x_1,x_2,label,day,prev_1_month,prev_1_month_day,prev_1_month_day_diff,prev_2_month,prev_2_month_day,prev_2_month_day_diff,prev_3_month,prev_3_month_day,prev_3_month_day_diff
date,i64,i64,i64,i8,date,i8,i8,date,i8,i8,date,i8,i8
2022-03-31,20556,27630,29201126524,31,2022-03-03,28,3,2022-01-31,31,0,2021-12-31,31,0
2022-03-30,21596,29330,26624140436,30,2022-03-02,28,2,2022-01-30,30,0,2021-12-30,30,0
2022-03-29,22929,31310,25605067518,29,2022-03-01,28,1,2022-01-29,29,0,2021-12-29,29,0
2022-03-28,29078,39085,30561931579,28,2022-02-28,28,0,2022-01-28,28,0,2021-12-28,28,0
2022-03-27,25968,34468,22647394491,27,2022-02-27,27,0,2022-01-27,27,0,2021-12-27,27,0
2022-03-26,31697,43999,28525252066,26,2022-02-26,26,0,2022-01-26,26,0,2021-12-26,26,0
2022-03-25,38359,52518,35312823287,25,2022-02-25,25,0,2022-01-25,25,0,2021-12-25,25,0
2022-03-24,44149,60934,36990483766,24,2022-02-24,24,0,2022-01-24,24,0,2021-12-24,24,0
2022-03-23,51485,71539,40465771440,23,2022-02-23,23,0,2022-01-23,23,0,2021-12-23,23,0


# Aggregated Lagged Values Features
- Rolling mean, max, min, median, standard deviation… of the same date in the previous 1, 2, 3… months or quarters or years....

In [67]:
# ====================================
# Rolling mean, max, min, median, standard deviation… of the same date in the previous 1, 2, 3… months or quarters or years....
for c in value_columns:
    df_prep = df_prep.with_columns(pl.concat_list(cs.starts_with(f"{c}_prev_")).list.mean().alias(f"{c}_same_date_rolling_mean"))
    df_prep = df_prep.with_columns(pl.concat_list(cs.starts_with(f"{c}_prev_")).list.min().alias(f"{c}_same_date_rolling_min"))
    df_prep = df_prep.with_columns(pl.concat_list(cs.starts_with(f"{c}_prev_")).list.max().alias(f"{c}_same_date_rolling_max"))
    df_prep = df_prep.with_columns(pl.concat_list(cs.starts_with(f"{c}_prev_")).list.std().alias(f"{c}_same_date_rolling_std"))

    # for easy showing result
    df_prep = df_prep.drop(cs.starts_with(f"{c}_prev_"))
        
# delete columns with name starts with "prev_"
df_prep = df_prep.drop(cs.starts_with("prev_"))

df_prep.head(10)

date,x_1,x_2,label,x_1_rolling_mean,x_1_rolling_min,x_1_rolling_max,x_1_rolling_std,x_2_rolling_mean,x_2_rolling_min,x_2_rolling_max,x_2_rolling_std,label_rolling_mean,label_rolling_min,label_rolling_max,label_rolling_std,x_1_same_date_rolling_mean,x_1_same_date_rolling_min,x_1_same_date_rolling_max,x_1_same_date_rolling_std,x_2_same_date_rolling_mean,x_2_same_date_rolling_min,x_2_same_date_rolling_max,x_2_same_date_rolling_std,label_same_date_rolling_mean,label_same_date_rolling_min,label_same_date_rolling_max,label_same_date_rolling_std
date,i64,i64,i64,f64,i64,i64,f64,f64,i64,i64,f64,f64,i64,i64,f64,f64,i64,i64,f64,f64,i64,i64,f64,f64,i64,i64,f64
2022-09-14,125490,169835,108957173922,90058.333333,77888,97057,10579.330902,122641.333333,109136,129753,11701.470093,7.8271e10,67757852224,84356083730,9.1424e9,90058.333333,77888,97057,10579.330902,122641.333333,109136,129753,11701.470093,7.8271e10,67757852224,84356083730,9.1424e9
2022-09-13,107326,146494,97902777699,88045.666667,82044,93640,5808.721403,119520.333333,113005,124479,5893.265422,7.8372e10,70168022431,83375464274,7.1619e9,88045.666667,82044,93640,5808.721403,119520.333333,113005,124479,5893.265422,7.8372e10,70168022431,83375464274,7.1619e9
2022-09-12,93820,130855,95690875998,82391.0,69934,88917,10792.179715,111725.666667,92317,123273,16909.136741,7.3986e10,55440756933,83972748818,1.6077e10,82391.0,69934,88917,10792.179715,111725.666667,92317,123273,16909.136741,7.3986e10,55440756933,83972748818,1.6077e10
2022-09-11,64274,87798,58820280634,73718.666667,69230,81485,6753.042302,100397.0,93500,114115,11880.197263,7.1797e10,61757119252,78554995379,8.8666e9,73718.666667,69230,81485,6753.042302,100397.0,93500,114115,11880.197263,7.1797e10,61757119252,78554995379,8.8666e9
2022-09-10,66959,90096,69852043855,80081.333333,52072,98153,24595.376686,107754.0,69508,134906,34081.25919,8.3556e10,52052998733,105589463477,2.7996e10,80081.333333,52072,98153,24595.376686,107754.0,69508,134906,34081.25919,8.3556e10,52052998733,105589463477,2.7996e10
2022-09-09,72821,97114,79781916805,62673.0,57879,65862,4226.593309,83871.666667,76511,89477,6658.842642,6.2166e10,60032826975,65070329562,2.6057e9,62673.0,57879,65862,4226.593309,83871.666667,76511,89477,6658.842642,6.2166e10,60032826975,65070329562,2.6057e9
2022-09-08,54406,71847,55782383425,61879.0,56290,67107,5417.528311,81357.333333,77199,85582,4191.893645,6.3511e10,60874604717,68754263224,4.5407e9,61879.0,56290,67107,5417.528311,81357.333333,77199,85582,4191.893645,6.3511e10,60874604717,68754263224,4.5407e9
2022-09-07,59809,75899,56484560972,44358.333333,36055,50476,7454.808135,57062.333333,48272,63996,8024.740017,4.4732e10,34110129387,50829170014,9.2322e9,44358.333333,36055,50476,7454.808135,57062.333333,48272,63996,8024.740017,4.4732e10,34110129387,50829170014,9.2322e9
2022-09-06,54644,69028,53346288660,42089.333333,38576,45581,3502.550261,54292.0,51831,57884,3181.049041,4.3482e10,37221762750,48609254247,5.7778e9,42089.333333,38576,45581,3502.550261,54292.0,51831,57884,3181.049041,4.3482e10,37221762750,48609254247,5.7778e9
